<h1> Классификация спама в e-mail сообщениях

<h2> Постановка задачи:</h2>
<p>Дан датасет - сообщения и пометка, является ли это сообщение спамом (957 сообщений на тренировочной выборке и 125 на тестовой). Цель задачи - научиться автоматически помечать сообщения со спамом.</p>

<h3> Читаем тренировочные данные

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import numpy as np          
import seaborn as sns         
import sklearn
import nltk 

train_dataset = pd.read_csv("archive/SMS_train.csv", sep=',', encoding="Windows-1252")

<h3> Взглянем на информацию о датасете

In [ ]:
train_dataset.shape

In [ ]:
train_dataset.head(10)

In [ ]:
train_dataset.tail(10)

In [ ]:
train_dataset.info()

<h3> Получаем набор всех слов встречающихся в выборке

In [ ]:
tokens = []
for xl in train_dataset["Message_body"]:
    token = nltk.word_tokenize(str(xl))
    for t in token:
        tokens.append(t)
tokens = set(tokens)

In [ ]:
#количество неповторяющихся слов
tokens.__len__()

<h3> Обрабатываем датасет. Теперь для каждой записи указано сколько раз встречается то или иное слово в сообщении

In [ ]:
from collections import Counter

df = train_dataset
for index, row in train_dataset.iterrows():
    count_letter = Counter(nltk.word_tokenize(row["Message_body"]))
    for token in tokens:
        df.loc[index, token] = count_letter[token]




df.drop("Message_body", axis=1, inplace=True)

In [ ]:
df.drop("S. No.", axis=1, inplace=True)

<h3> Вот так теперь выглядит датасет

In [ ]:
df.head(10)

In [ ]:
df.info()